# LAB 3 - UART integration

In [142]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

/dev/ttyUSB1 - USB Serial
/dev/ttyUSB3 - TUL - TUL


In [143]:
import serial
ser = serial.Serial('/dev/ttyUSB3', 115200, timeout=10)

First, create two random operands (A,B) that have OPERAND_WIDTH length.

In [144]:
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

if A < B:
    A, B = B, A

resAdd = A + B
resSub = A - B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(resAdd))
print("A - B = ", hex(resSub))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

A     =  0xa547004bd21ac80b2df2b4eb92b7ca982e134fbcb963422104aadee33f6567a7787e49146701038e0adbc232bb2d3f7204a8721bd451ccc8d5d28b1b9075c690
B     =  0x871b340429bbc1f60b85ae16ee1d02fa571a012aafa44edfde446ae80b9e0e3c267cd2fe69a0ad6a10a6fb3aafb363c2570ab0db595410d5979d29d0a84bb618
A + B =  0x12c62344ffbd68a013978630280d4cd92852d50e769079100e2ef49cb4b0375e39efb1c12d0a1b0f81b82bd6d6ae0a3345bb322f72da5dd9e6d6fb4ec38c17ca8
A - B =  0x1e2bcc47a85f0615226d06d4a49ac79dd6f94e9209bef341266673fb33c7596b52017615fd605623fa34c6f80b79dbafad9dc1407afdbbf33e35614ae82a1078


Then, send the two operands (A,B) via the serial port, byte by byte. Later, the PYNQ board receives the result of the addition, and transmits it back to the computer.

In [ ]:
for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)
    
    
op = 0x00
ser.write(bytes([op]))
time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

print("Operation: ",hex(op))

if res2==resAdd:
    print("Result received correctly!")
    print("Expected   = ", hex(resAdd))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(resAdd))
    print("Received   = ", hex(res2))
    print("Offset:   = ", hex(res2-resAdd))



In [141]:
ser.close()